# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [87]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [88]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [89]:
data.head(30)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [90]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [91]:
data_ultra = data[data['is_ultra']==0]
data_ultra.shape

(2229, 5)

Две трети пользователей, пользуются тарифом ультра.

## Разбейте данные на выборки

In [92]:
#выделим признаки и цель
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']
#введем random_state как постоянную величину, что бы не ошибиться
random_state=12345
#разобьем на обучающую, валидационную и тестовую выборки в соотношении 3:1:1
features, features_test, target, target_test = train_test_split(
    features, target, test_size=0.2, random_state=random_state, stratify=target)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, train_size=0.25, random_state=random_state, stratify=target)


In [93]:
print('train',features_train.shape,target_train.shape)
print('test',features_test.shape,target_test.shape)
print('valid',features_valid.shape,target_valid.shape)

train (642, 4) (642,)
test (643, 4) (643,)
valid (1929, 4) (1929,)


## Исследуйте модели

**Т.к. поставленная задача соответсвует задачи классификации будем использовать следущие модели:**

* Логистическая регрессия
* Дерево решений
* Случайный лес


In [94]:
best_model_lr = None
best_result_lr = 0
for solver in ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']:
    for max_iter in [100,200,300,400,500,1000]:
        model = LogisticRegression(random_state=random_state, solver=solver, max_iter=max_iter) 
        model.fit(features_train,target_train) 
        result = model.score(features_valid,target_valid)
        if result > best_result_lr:
            best_model_lr = model
            best_result_lr = result
print(best_model,best_result)

/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorith

LogisticRegression(random_state=12345, solver='newton-cg') 0.7304302747537584


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


 **Логистическая регрессия** :Модель не показала, необходимого результата и к расмотрению не допускается.

In [95]:
best_model_dtc = None
best_result_dtc = 0
results_dtc_tr = []
results_dtc_val = []
for min_samples_split in range (2,5):
    for splitter in ['best','random']:
        for criterion in ['gini','entropy']:
            for max_depth in range(1, 20):
                model = DecisionTreeClassifier(random_state=random_state, max_depth=max_depth,criterion=criterion,\
                                   splitter=splitter,min_samples_split=min_samples_split) 
                model.fit(features_train,target_train) 
                result = model.score(features_valid,target_valid)
                if result > best_result_dtc:
                    best_model_dtc = model
                    best_result_dtc = result
print(best_model_dtc,best_result_dtc)

DecisionTreeClassifier(max_depth=5, random_state=12345) 0.7947122861586314


 **Дерево решений**:
 Модель показала необходимую точность, рассмотрим при каких значениях гиперпараметров.

In [96]:
print(best_model_dtc.get_params())

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 12345, 'splitter': 'best'}


**Значения гиперпараметров решающего дерева**
* criterion - gini
* min_samples_split - 2
* splitter - best
* max_depth - 5

In [97]:
best_model_rfc = None
best_result_rfc = 0
for criterion in ['gini', 'entropy']:
    for max_features in ['sqrt', 'log2']:
        for n_estimators in range(5,15):
            for max_depth in range(5,15):
                model = RandomForestClassifier(random_state=random_state, criterion=criterion,
                                    max_features=max_features, max_depth=max_depth, n_estimators=n_estimators)
                model.fit(features_train,target_train) 
                result = model.score(features_valid,target_valid)
                if result > best_result_rfc:
                    best_model_rfc = model
                    best_result_rfc = result
print(best_model_rfc,best_result_rfc)

RandomForestClassifier(max_depth=8, max_features='sqrt', n_estimators=10,
                       random_state=12345) 0.8009331259720062


 **Случайный лес**:
 Модель так же показала необходимую точность, рассмотрим при каких значениях гиперпараметров.

In [98]:
print(best_model_rfc.get_params())


{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 8, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 10, 'n_jobs': None, 'oob_score': False, 'random_state': 12345, 'verbose': 0, 'warm_start': False}


**Значения гиперпараметров случайного леса**
* criterion - gini
* n_estimators - 10
* max_depth - 8
* max_features - sqrt

**Красткий вывод**: т.к. accuracy  решающего древа и случайного леса практически не отличаются, проверим на тестовой выборке обе модели.

## Проверьте модель на тестовой выборке

Посмотрим результат случайного леса на тестовой выборке.

In [99]:
result = best_model_rfc.score(features_test,target_test)
print(result)

0.7962674961119751


Посмотрим результат обучающего древа на тестововй выборке.

In [100]:
result = best_model_dtc.score(features_test,target_test)
print(result)

0.7962674961119751


**Вывод**: т.к. результаты совпали лучше использовать решающее дерево.

Попробуем еще раз обучить модоели на слиянии тренировочной и валидационной выборок.

In [105]:
features_train_2 = pd.concat([features_train,features_valid])
target_train_2 = pd.concat([target_train,target_valid])
model_rfc = RandomForestClassifier(random_state=random_state, criterion='gini',
                                    max_features='sqrt', max_depth=8, n_estimators=10)
model_rfc.fit(features_train_2,target_train_2) 
result_rfc = model.score(features_valid,target_valid)
model_dtc = DecisionTreeClassifier(random_state=random_state, max_depth=5,criterion='gini',\
                                   splitter='best',min_samples_split=2) 
model_dtc.fit(features_train_2,target_train_2) 
result_dtc = model.score(features_train_2,target_train_2)
print(result_rfc,result_dtc)

0.7807153965785381 0.8265266433294438


Подтвердили вывод, что решающее дерево справилось лучше.

## (бонус) Проверьте модели на адекватность

Поскольку в предоставленных для обучения сданных большинство пользователей пользовалось тарифом ультра можно предположить, что если всегда его предлагать то в 2/3 случаев констаннтная модель будет права, проверим.

In [101]:
const_all_ultra = pd.Series ([0] * len(target_test))
result = accuracy_score(target_test, const_all_ultra)
print(result)


0.6936236391912908


Подозрительно высокий результат, который не сильно уступает обученной модели, можно сделать вывод, что диссбаланс в данных сильно портит обучение модели и следовательно оправданность ее применения в бизнесе.